# SQL Database for Blockbuster

*Commit 1*

In [1]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)

import mysql.connector as conn

## 1. Create new database called blockbuster and connect to it

In [2]:
conexion = conn.connect(host='localhost',
                        user='root',
                        passwd='password'
                       )

cursor = conexion.cursor()


cursor

In [3]:
cursor.execute('drop database if exists blockbuster;')

cursor.execute('create database blockbuster;')

conexion = conn.connect(host='localhost',
                        user='root',
                        passwd='password',
                        database='blockbuster'
                       )


cursor = conexion.cursor()

c=cursor.execute

c


<bound method MySQLCursor.execute of <mysql.connector.cursor.MySQLCursor object at 0x7fb36b6fd720>>

## 2. Look at the Data

In [4]:
actor = pd.read_csv('../data/actor.csv')
categ = pd.read_csv('../data/category.csv')
film = pd.read_csv('../data/film.csv')
invent = pd.read_csv('../data/inventory.csv')
lang = pd.read_csv('../data/language.csv')
old = pd.read_csv('../data/old_HDD.csv')
rental = pd.read_csv('../data/rental.csv')

*Commit 2,3 y 4*

## 3. Data Cleaning

### Deleting update time from all the columns

In [9]:
x='last_update'

actor.drop(x, axis=1, inplace= True)
lang.drop(x, axis=1, inplace= True)
rental.drop(x, axis=1, inplace= True)
invent.drop(x, axis=1, inplace= True)
film.drop(x, axis=1, inplace= True)
categ.drop(x, axis=1, inplace= True)

### Invent/Films fix

In [10]:
film.drop('original_language_id', axis=1, inplace= True)

### Deleting duplicated actor

In [11]:
actor.drop(actor[actor.duplicated(subset=['first_name', 'last_name'])].index, inplace=True)

### Changing the actor names from 'old' to actor_id

In [12]:
old['actor_id'] = pd.Series([np.nan] * len(old))
old['full_name'] = old['first_name'] + ' ' + old['last_name']

del old['first_name']
del old['last_name']



In [13]:
actor['full_name'] = actor['first_name'] + ' ' + actor['last_name']

del actor['first_name']
del actor['last_name']


*commit 5*

In [14]:
actores_ides = dict(zip(actor.full_name.values,actor.actor_id.values))


In [15]:
old['actor_id'] = [actores_ides[i] for i in old.full_name]

In [16]:
film_ides = dict(zip(film.title.values,film.film_id.values))

In [17]:
old['title_id'] = [film_ides[i] for i in old.title]

In [18]:
bad_columns= ['release_year', 'full_name', 'title']

old.drop(columns=bad_columns, inplace=True)

,category_id,actor_id,title_id
0,6,1,1
1,2,1,23
2,13,1,25
3,10,1,106
4,14,1,140
...,...,...,...
995,3,39,168
996,14,39,203
997,11,39,222
998,9,39,290


*Commit 6*

### Category  to film table

In [19]:
film.drop(film[film.duplicated(subset=['title'])].index, inplace=True)

In [20]:
selected_columns = old[['category_id', 'title_id']]
tuples = [tuple(row) for row in selected_columns.itertuples(index=False)]
unique_tuples = set(tuples)

In [21]:
def categ_sust(x):
    for a, b in unique_tuples:
        if x == b:
            return int(a)
    return None

In [22]:
film['category_id'] = film['film_id'].apply(categ_sust)

In [23]:
nan_rows = film[film.isnull().any(axis=1)]
nan_rows.shape
film.fillna(0, inplace=True)

In [24]:
film['category_id'] = film['category_id'].astype(int)

In [25]:
old.drop('category_id', axis=1, inplace= True)

### Delete store id from inventory

In [26]:
invent.drop(columns='store_id', inplace=True)